## Импорт нужных модулей

In [10]:
from pymystem3 import Mystem
m = Mystem()
import json
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import nltk
from nltk.tokenize import word_tokenize
import jsonlines
import collections as c
import pandas as pd

## Пункт 2
Обработать книгу с майстем
+ открываем и читаем файл
+ лемматизируем и засекаем время
+ записываем в файл

In [ ]:
with open ('mumu.txt', encoding = 'utf-8') as f:
    big_text = f.read()
%time lemmas = m.lemmatize(big_text)
with open('lemmas.json', 'a', encoding='utf-8') as j:
    json.dump(lemmas, j, ensure_ascii = False)

## Пункт 3
Обработать книгу с pymorphy
+ токенизируем и засекаем время
+ запускаем цикл, в котором:
1. для каждого слова делаем разбор
2. выбираем нужную информацию
3. записываем е в файл

In [33]:
%time tok = word_tokenize(big_text)
for w in tok:
    p = morph.parse(w)[0]
    #my_tag = p.tag.POS
    #print(my_tag)
    info = {'lemma' : p.normal_form, 'word': p.word, 'pos': p.tag.POS}
    #print(info)
    with jsonlines.open('info.jsonl', mode='a') as writer:
        writer.write(info)

Wall time: 64.6 ms


? что это такое ?

In [27]:
%time tok = word_tokenize(big_text)
lemmas = []
for w in tok:
    p = morph.parse(w)[0]
    lemmas.append(list(p))

Wall time: 87.2 ms


## Пункт 4
Какую долю слов составляет каждая часть речи?
+ создаём словарь, где ключ — часть речи, значение — список слов
+ считаем общее число слов
+ считаем процент и делаем список списков
+ строим таблицу

In [46]:
pos = c.defaultdict(list)
summ = 0
my_data = []
for w in tok:
    p = morph.parse(w)[0]
    part = str(p.tag.POS)
    pos[part].append(w)
    summ += 1
for p in sorted(pos.keys()):
    perc = (len(pos[p])/summ)*100
    my_data.append([p, perc])
df = pd.DataFrame(my_data, columns=["POS", "Percentage"])
df

,POS,Percentage
0,ADJF,7.527184
1,ADJS,0.646794
2,ADVB,5.614923
3,COMP,0.168729
4,CONJ,7.649044
5,GRND,0.853018
6,INFN,1.593551
7,INTJ,0.149981
8,NOUN,18.907012
9,NPRO,7.367829


Найдите топ-20 (по частотности) глаголов и наречий
+ вычленяем список всех глаголов из словаря
+ делаем частотный словарь и печатаем его
+ то же для наречий

In [7]:
verb = pos['VERB']
count_v = c.Counter(verb)
print(count_v.most_common(20))
adverb = pos['ADVB']
count_a = c.Counter(adverb)
print(count_a.most_common(20))

[('было', 34), ('был', 25), ('была', 10), ('вышел', 9), ('начал', 8), ('глядел', 7), ('продолжал', 7), ('мог', 6), ('стал', 6), ('пошел', 6), ('посмотрел', 6), ('подумал', 6), ('возразил', 6), ('вернулся', 6), ('хотел', 5), ('поставил', 5), ('убьет', 5), ('заметил', 5), ('видел', 5), ('отправился', 5)]
[('только', 39), ('опять', 19), ('вдруг', 19), ('уже', 18), ('потом', 16), ('очень', 16), ('наконец', 13), ('тотчас', 13), ('теперь', 8), ('никогда', 8), ('раз', 8), ('сперва', 8), ('тут', 8), ('вместе', 8), ('по-прежнему', 7), ('где', 6), ('несколько', 6), ('назад', 6), ('вовсе', 5), ('вон', 5)]


## Пункт 5
топ-25 биграмм и триграмм
+ приводим слова к нижнему регистру, убираем знаки препинания
+ ищем биграммы и изменяем их в список
+ делаем частотный словарь и печатаем его
+ то же для триграмм

In [9]:
words = [w.lower() for w in tok if w.isalpha()]
wordsbigrm = nltk.bigrams(words)
bg = list(wordsbigrm)
cb = c.Counter(bg)
wordstrigrm = nltk.trigrams(words)
tg = list(wordstrigrm)
ct = c.Counter(tg)
print(cb.most_common(25), ct.most_common(25))

[(('гаврила', 'андреич'), 16), (('у', 'него'), 13), (('и', 'не'), 10), (('как', 'бы'), 10), (('у', 'ней'), 10), (('на', 'него'), 9), (('на', 'нее'), 9), (('в', 'доме'), 9), (('к', 'себе'), 9), (('к', 'ней'), 9), (('с', 'ним'), 8), (('на', 'дворе'), 8), (('так', 'и'), 7), (('только', 'что'), 6), (('и', 'он'), 6), (('а', 'что'), 6), (('как', 'будто'), 6), (('в', 'свою'), 6), (('не', 'было'), 6), (('ведь', 'он'), 6), (('ну', 'да'), 6), (('на', 'себя'), 6), (('что', 'он'), 6), (('и', 'только'), 6), (('вместе', 'с'), 6)] [(('в', 'свою', 'каморку'), 5), (('себя', 'в', 'грудь'), 3), (('на', 'другой', 'день'), 3), (('в', 'этом', 'случае'), 3), (('к', 'себе', 'в'), 3), (('герасим', 'вышел', 'из'), 3), (('на', 'другое', 'утро'), 3), (('метлу', 'и', 'лопату'), 2), (('у', 'него', 'было'), 2), (('было', 'готово', 'и'), 2), (('на', 'дворе', 'у'), 2), (('у', 'него', 'как'), 2), (('во', 'все', 'стороны'), 2), (('а', 'то', 'беда'), 2), (('всё', 'же', 'он'), 2), (('на', 'левой', 'щеке'), 2), (('у', 'ней